In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


AutoML SDK: AutoML自动化视频对象跟踪模型

安装

安装最新（预览）版本的AutoML SDK。

In [ ]:
! pip3 install -U google-cloud-automl --user


安装谷歌云存储库。

In [ ]:
! pip3 install google-cloud-storage


### 重新启动内核

一旦您安装了AutoML SDK和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os


if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


## 开始之前

### GPU 运行时

*如果您有这个选项，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 GCP 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 GCP 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得 $300 的免费信用额，用于付费计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 AutoML API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 AutoML 笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 会将以 `!` 为前缀的行作为 shell 命令运行，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}


In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)


In [ ]:
! gcloud config set project $PROJECT_ID


#### 地区

您也可以更改“REGION”变量，该变量用于本笔记本其余部分的操作。以下是AutoML支持的地区。我们建议尽可能选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您不能使用多区域存储存储桶进行AutoML训练。并非所有地区都支持所有AutoML服务。有关每个地区最新的支持情况，请参见[AutoML服务的地区支持]()。

In [ ]:
REGION = 'us-central1' #@param {type: "string"}


时间戳

如果您正在参加实况教程会话，可能会使用共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并追加到将在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime


TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")


### 验证您的GCP账户

**如果您正在使用AutoML笔记本**，您的环境已经验证通过。请跳过这一步。

*注意：如果您在AutoML笔记本上运行代码单元格，则单元格会自动跳过执行验证步骤。*

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AutoML, then don't execute this code
if not os.path.exists('/opt/deeplearning/metadata/env_version'):
    if 'google.colab' in sys.modules:
        from google.colab import auth as google_auth
        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login


### 创建一个云存储桶

**无论您使用什么笔记本环境，都需要按照以下步骤进行。**

本教程旨在使用公共云存储桶中的训练数据以及本地云存储桶用于批量预测。您也可以使用您自己存储在本地云存储桶中的训练数据。

请在下方设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}


In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP


只有在您的存储桶尚不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME


最后，通过检查云存储桶的内容来验证访问权限：

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME


### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

#### 导入 AutoML SDK

将 AutoML SDK 导入到我们的 Python 环境中。

In [ ]:
import json
import os
import sys
import time


from google.cloud import automl_v1beta1 as automl


from google.protobuf.json_format import MessageToJson
from google.protobuf.json_format import ParseDict



AutoML常量

为AutoML设置以下常量：

- `PARENT`：用于数据集、模型和端点资源的AutoML位置根路径。

In [ ]:
# AutoML location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION


客户端

AutoML SDK 采用客户端/服务器模型。在您的一侧 (Python 脚本) 中，您将创建一个客户端，该客户端会向服务器 (AutoML) 发送请求并接收响应。

在本教程中，您将使用多个客户端，因此请提前配置好它们。

- 数据集服务用于管理数据集。
- 模型服务用于管理模型。
- 管道服务用于训练。
- 终端服务用于部署。
- 作业服务用于批处理作业和定制训练。
- 预测服务用于提供服务。*注意*: 预测具有不同的服务端点。

In [ ]:
def automl_client():
    return automl.AutoMlClient()

def prediction_client():
    return automl.PredictionServiceClient()

def operations_client():
    return automl.AutoMlClient()._transport.operations_client


clients = {}
clients["automl"] = automl_client()
clients["prediction"] = prediction_client()
clients["operations"] = operations_client()

for client in clients.items():
    print(client)


In [ ]:
IMPORT_FILE = 'gs://automl-video-demo-data/traffic_videos/traffic_videos.csv'


In [ ]:
! gsutil cat $IMPORT_FILE | head -n 10


未分配, gs://automl-video-demo-data/traffic_videos/traffic_videos_labels.csv

## 创建数据集

### [projects.locations.datasets.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/create)

将以上英文文本翻译为中文：### [projects.locations.datasets.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/create)

请求

In [ ]:
dataset = {
    "display_name": "traffic_" + TIMESTAMP,
    "video_object_tracking_dataset_metadata": {}
}

print(MessageToJson(
    automl.CreateDatasetRequest(
        parent=PARENT,
        dataset=dataset
    ).__dict__["_pb"])
)


*示例输出*：
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "traffic_20210310004803",
    "videoObjectTrackingDatasetMetadata": {}
  }
}
```

*示例输出*：
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "traffic_20210310004803",
    "videoObjectTrackingDatasetMetadata": {}
  }
}
```

呼叫

In [ ]:
request = clients["automl"].create_dataset(
    parent=PARENT,
    dataset=dataset
)


#### 回应

In [ ]:
result = request

print(MessageToJson(result.__dict__["_pb"]))


```
{
  "name": "projects/116273516712/locations/us-central1/datasets/VOT4951119001817186304",
  "displayName": "traffic_20210310004803",
  "createTime": "2021-03-10T00:48:09.292248Z",
  "etag": "AB3BwFp3u4a2oy-k3EgK6ci8zwrTqrd91_DmoaY8TYsxnb-N-aXwFefqCIm1z0YTM290",
  "videoObjectTrackingDatasetMetadata": {}
}
```

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split('/')[-1]

print(dataset_id)


### [项目.位置.数据集.导入数据](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/importData)

请求

In [ ]:
input_config = {
    "gcs_source": {
        "input_uris": [IMPORT_FILE]
    }
}

print(MessageToJson(
    automl.ImportDataRequest(
        name=dataset_short_id,
        input_config=input_config
    ).__dict__["_pb"])
)


```
{
  "name": "VOT4951119001817186304",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://automl-video-demo-data/traffic_videos/traffic_videos.csv"
      ]
    }
  }
}
```
```
{
  "名称": "VOT4951119001817186304",
  "输入配置": {
    "gcsSource": {
      "inputUris": [
        "gs://automl-video-demo-data/traffic_videos/traffic_videos.csv"
      ]
    }
  }
}
```

电话

In [ ]:
request = clients["automl"].import_data(
    name=dataset_id,
    input_config=input_config
)


#### 回复

In [ ]:
result = request.result()

print(MessageToJson(result))


*示例输出*：
```
{}
```

训练一个模型

### [projects.locations.models.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/create)

### [项目.位置.模型.创建](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/create)

### 请求

In [ ]:
model = {
    "display_name": "traffic_" + TIMESTAMP,
    "dataset_id": dataset_short_id,
    "video_object_tracking_model_metadata": {}
}

print(MessageToJson(
    automl.CreateModelRequest(
        parent=PARENT,
        model=model
    ).__dict__["_pb"])
)


*示例输出*：
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "model": {
    "displayName": "traffic_20210310004803",
    "datasetId": "VOT4951119001817186304",
    "videoObjectTrackingModelMetadata": {}
  }
}
``` 

*示例输出*：
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "model": {
    "displayName": "traffic_20210310004803",
    "datasetId": "VOT4951119001817186304",
    "videoObjectTrackingModelMetadata": {}
  }
}

电话

In [ ]:
request = clients["automl"].create_model(
    parent=PARENT,
    model=model
)


#### 回复

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


{
  "name": "projects/116273516712/locations/us-central1/models/VOT6634816000837550080"
}
{
  "name": "projects/116273516712/locations/us-central1/models/VOT6634816000837550080"
}

In [ ]:
# The full unique ID for the training pipeline
model_id = result.name
# The short numeric ID for the training pipeline
model_short_id = model_id.split('/')[-1]

print(model_short_id)


评估模型

### [projects.locations.models.modelEvaluations.list](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/list)

### [projects.locations.models.modelEvaluations.list](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/list)

#### 电话

In [ ]:
request = clients["automl"].list_model_evaluations(
    parent=model_id, 
)


#### 回复

In [ ]:
for evaluation in request:
    print(MessageToJson(evaluation.__dict__["_pb"]))

# The last evaluation slice
last_evaluation_slice = evaluation.name


*示例输出*：
{
  "name": "projects/116273516712/locations/us-central1/models/VOT6634816000837550080/modelEvaluations/3861662240237183015",
  "annotationSpecId": "7080515133784457216",
  "createTime": "2021-03-10T01:57:44.615737Z",
  "evaluatedExampleCount": 6,
  "videoObjectTrackingEvaluationMetrics": {
    "boundingBoxMetricsEntries": [
      {
        "iouThreshold": 0.5,
        "meanAveragePrecision": 0.30026233,
        "confidenceMetricsEntries": [
          {
            "recall": 1.0,
            "precision": 0.37222221,
            "f1Score": 0.5425101
          },
          {
            "confidenceThreshold": 0.028951555,
            "recall": 0.13432837,
            "precision": 0.07377049,
            "f1Score": 0.0952381
          },
          
          # 省略部分内容
          
          {
            "confidenceThreshold": 1.0,
            "precision": 1.0
          }
        ]
      }
    ],
    "boundingBoxMeanAveragePrecision": 0.30026233
  },
  "displayName": "pickup_suv_van"
}
{
  "name": "projects/116273516712/locations/us-central1/models/VOT6634816000837550080/modelEvaluations/4053501621797068779",
  "annotationSpecId": "5927593629177610240",
  "createTime": "2021-03-10T01:57:44.615737Z",
  "evaluatedExampleCount": 5,
  "videoObjectTrackingEvaluationMetrics": {
    "boundingBoxMetricsEntries": [
      {
        "iouThreshold": 0.5,
        "meanAveragePrecision": 0.42889464,
        "confidenceMetricsEntries": [
          {
            "recall": 1.0,
            "precision": 0.25490198,
            "f1Score": 0.40625
          },
          
          # 省略部分内容
    ],
    "boundingBoxMeanAveragePrecision": 0.34359422
  },
  "displayName": "sedan"
}

### [projects.locations.models.modelEvaluations.get](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/get)

### [projects.locations.models.modelEvaluations.get](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/get)

呼叫

In [ ]:
request = clients["automl"].get_model_evaluation(
    name=last_evaluation_slice
)


回应

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


```
{
  "name": "projects/116273516712/locations/us-central1/models/VOT6634816000837550080/modelEvaluations/6603022638609369541",
  "annotationSpecId": "1315907610750222336",
  "createTime": "2021-03-10T01:57:44.615737Z",
  "evaluatedExampleCount": 6,
  "videoObjectTrackingEvaluationMetrics": {
    "boundingBoxMetricsEntries": [
      {
        "iouThreshold": 0.5,
        "meanAveragePrecision": 0.34359422,
        "confidenceMetricsEntries": [
          {
            "recall": 1.0,
            "precision": 0.41428572,
            "f1Score": 0.5858586
          },
          {
            "confidenceThreshold": 0.03328514,
            "recall": 0.1724138,
            "precision": 0.10869565,
            "f1Score": 0.13333334
          },
          
          # 略
        ]
      }
    ],
    "boundingBoxMeanAveragePrecision": 0.34359422
  },
  "displayName": "sedan"
}
```

## 进行批量预测

### 准备批量预测数据

In [ ]:
TRAIN_FILES = "gs://automl-video-demo-data/traffic_videos/traffic_videos_labels.csv"

test_items = ! gsutil cat $TRAIN_FILES | head -n2

cols = str(test_items[0]).split(',')
test_item_1 = str(cols[0])
test_label_1 = str(cols[1])
test_start_time_1 = str(0)
test_end_time_1 = "inf"

print(test_item_1, test_label_1)

cols = str(test_items[1]).split(',')
test_item_2 = str(cols[0])
test_label_2 = str(cols[1])
test_start_time_2 = str(0)
test_end_time_2 = "inf"

print(test_item_2, test_label_2)


示例输出：
```
gs://automl-video-demo-data/traffic_videos/highway_005.mp4 轿车
gs://automl-video-demo-data/traffic_videos/highway_005.mp4 皮卡SUV面包车
```

### 创建批量输入文件

要从AutoML Video请求一批预测结果，请创建一个CSV文件，列出您想要注释的视频在Cloud Storage中的路径。您还可以指定开始和结束时间，告诉AutoML Video只对视频的一部分（分段级别）进行注释。开始时间必须为零或更大，且必须在结束时间之前。结束时间必须大于开始时间且小于或等于视频的持续时间。您还可以使用inf来表示视频的结尾。

In [ ]:
import tensorflow as tf
import json

gcs_input_uri = "gs://" + BUCKET_NAME + '/test.csv'
with tf.io.gfile.GFile(gcs_input_uri, 'w') as f:
    data = f"{test_item_1}, {test_start_time_1}, {test_end_time_1}"
    f.write(data + '\n')
    data = f"{test_item_2}, {test_start_time_2}, {test_end_time_2}"
    f.write(data + '\n')
    
print(gcs_input_uri)
!gsutil cat $gcs_input_uri


以下是例子输出：
```
gs://migration-ucaip-trainingaip-20210310004803/test.csv
gs://automl-video-demo-data/traffic_videos/highway_005.mp4, 0, inf
gs://automl-video-demo-data/traffic_videos/highway_005.mp4, 0, inf
```

### [projects.locations.models.batchPredict](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/batchPredict)

### [项目位置模型批量预测](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/batchPredict)

#### 请求

In [ ]:
input_config = {
    "gcs_source": {
        "input_uris": [gcs_input_uri]
    }
}
    
output_config = {
    "gcs_destination": {
        "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"
    }
}                          

batch_prediction = automl.BatchPredictRequest(
    name=model_id,
    input_config=input_config,
    output_config=output_config,
)

print(MessageToJson(batch_prediction.__dict__["_pb"]))


{
  "name": "projects/116273516712/locations/us-central1/models/VOT6634816000837550080",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://migration-ucaip-trainingaip-20210310004803/test.csv"
      ]
    }
  },
  "outputConfig": {
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210310004803/batch_output/"
    }
  }
}
```

*示例输出*：

拨打

In [ ]:
request = clients["prediction"].batch_predict(
    request=batch_prediction
)


回答

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


示例输出：

```
{}
```

In [ ]:
destination_uri = batch_prediction.output_config.gcs_destination.output_uri_prefix[:-1]

! gsutil ls $destination_uri/prediction-**
! gsutil cat $destination_uri/prediction-**

*示例输出*：
```
gs://migration-ucaip-trainingaip-20210310004803/batch_output/prediction-traffic_20210310004803-2021-03-10T01:57:52.257240Z/highway_005_1.json
gs://migration-ucaip-trainingaip-20210310004803/batch_output/prediction-traffic_20210310004803-2021-03-10T01:57:52.257240Z/video_object_tracking.csv
{
  "object_annotations": [ {
    "annotation_spec": {
      "display_name": "sedan",
      "description": "sedan"
    },
    "confidence": 0.52724433,
    "frames": [ {
      "normalized_bounding_box": {
        "x_min": 0.27629745,
        "y_min": 0.59244406,
        "x_max": 0.53941643,
        "y_max": 0.77127469
      },
      "time_offset": {
      
      }
    }, {
      "normalized_bounding_box": {
        "x_min": 0.135607,
        "y_min": 0.58437037,
        "x_max": 0.42441425,
        "y_max": 0.77325606
      },
      "time_offset": {
        "nanos": 100000000
      }
    }, 
    
    # 由于篇幅原因已删除
    
    }
  } ]
}
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,0,315576000000,gs://migration-ucaip-trainingaip-20210310004803/batch_output/prediction-traffic_20210310004803-2021-03-10T01:57:52.257240Z/highway_005_1.json,OK
```

清理工作

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除您在本教程中创建的各个资源。

In [ ]:
delete_dataset = True
delete_model = True
delete_bucket = True

# Delete the dataset using the AutoML fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients['automl'].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the AutoML fully qualified identifier for the model
try:
    if delete_model:
        clients['automl'].delete_model(name=model_id)
except Exception as e:
    print(e)

if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r gs://$BUCKET_NAME
